In [1]:
%matplotlib notebook
import csv
from scipy.io import loadmat
import numpy as np
import scipy.optimize as opt
from pandas.io.parsers import read_csv
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from sklearn.preprocessing import PolynomialFeatures

In [2]:
data = loadmat('ex4data1.mat')
y = data['y']
x = data['X']
unos = np.full((5000,1),1)
weights = loadmat('ex4weights.mat')
theta1, theta2 = weights['Theta1'], weights['Theta2']
x = np.append(unos, x, axis=1)
params = np.concatenate((np.ravel(theta1),np.ravel(theta2)))
theta1.shape

(25, 401)

In [3]:
def sigmoide(z):
    s = np.dot(z,-1)
    e = np.exp(s)
    d = 1 + e
    return 1/d

In [4]:
def propagacion (x,th):
  
    unos = np.full((len(x),1),1)
    z = np.dot(x,th)
    return np.append(unos,sigmoide(z), axis = 1)

In [5]:
def testeo(y, f):
    acertados = 0
    for i in range(len(y)):
        if np.argmax(f[i])+1 == y[i]:
            acertados +=1
    return acertados, acertados/len(y)*100

In [6]:
def norm(y,num_etiqueta):
    y_norm = np.zeros((len(y),num_etiqueta))
    for i in range(len(y)):
        y_norm[i][y[i]-1]= 1
    return np.transpose(y_norm)

In [7]:
def coste(theta1,theta2,x,y,num_etiqueta, reg):
    y = norm(y,num_etiqueta)
    g = np.copy(propagacion(propagacion(x,theta1.T),theta2.T)[:,1:])
    t1 = np.sum(np.multiply(-y.T,np.log(g)) - np.multiply((1-y.T),np.log(1-g)))/len(x)
    t2 = (np.sum(theta1**2) + np.sum(theta2**2))*reg/(2*len(x))
    return t1+t2

In [8]:
def backprop (params_rn, num_entradas, num_ocultas, num_etiquetas, x, y, reg = 1):
    theta1 = np.reshape(params_rn[:num_ocultas*(num_entradas+1)], (num_ocultas, (num_entradas+1)))
    theta2 = np.reshape(params_rn[num_ocultas*(num_entradas+1):], (num_etiquetas, (num_ocultas+1)))
    cost = coste(theta1,theta2,x,y,num_etiquetas, reg)
    return cost

In [9]:
backprop(params,theta1.shape[1]-1,theta2.shape[1]-1,10,x,y)

0.384487796242894

In [10]:
def deriv_sig(z):
    return np.multiply(sigmoide(z),1-sigmoide(z))

In [29]:
def pesosAleatorios(L_in, L_out, e=0.12):
    return 2*e*np.random.random_sample((L_out, L_in)) - e

In [30]:
thr1 = pesosAleatorios(theta1.shape[0], theta1.shape[1])
thr2 = pesosAleatorios(theta2.shape[0], theta2.shape[1])
thr1.shape

(401, 25)

In [138]:
def retro_propagacion(x,y,th1,th2):
    delta = []
    a2 = propagacion (x,th1)
    a3 = propagacion (a2,th2)[:,1:]
    d3 = a3-norm(y,10).T
    d2 = np.multiply(np.dot(th2,d3.T).T,deriv_sig(sigmoide(a2)))
    delta.append(np.sum((th1[1:] + np.dot(d2.T,x[:,1:])[1:].T)/len(x)))
    delta.append(np.sum((th2[1:] + np.dot(d3.T,a2)[:,1:].T/len(x))))
    return delta

In [139]:
propagacion(propagacion(x,thr1),thr2)[:,1:].shape

(5000, 10)

In [142]:
retro_propagacion(x,y,thr1,thr2)[1]

54.181534179890704

In [119]:
a2 = propagacion (x,thr1)
a3 = propagacion (a2,thr2)[:,1:]
d3 = a3-norm(y,10).T
np.dot(thr2,d3.T).T.shape
deriv_sig(sigmoide(a2)).shape
d2 = np.multiply(np.dot(thr2,d3.T).T,deriv_sig(sigmoide(a2)))
d2.shape
a2.shape

(5000, 26)